<span style="background-color: rgb(255, 255, 254);">F</span><span style="background-color: rgb(255, 255, 255); font-size: 11pt; font-family: ArialMT;">ind the info</span> <span style="background-color: rgb(255, 255, 255); font-size: 11pt; font-family: ArialMT;">of each buyers' most recent purchase</span> <span style="background-color: rgb(255, 255, 255); font-family: ArialMT; font-size: 14.6667px;">in 2021</span>

In [ ]:
SELECT p1.Buyer_ID, p1.Purchase_ID, p1.Purchase_Date, p1.Purchase_Amount 
FROM Purchases_Project..Purchases p1 
JOIN (SELECT Buyer_ID, MAX(DAY(Purchase_Date)) AS maxday, 
      MAX(MONTH(Purchase_Date)) AS maxmonth
      FROM Purchases_Project..Purchases
      GROUP BY Buyer_ID) p2
    ON p1.Buyer_Id = p2.Buyer_ID 
--make sure that all elements of the date match
WHERE DAY(p1.Purchase_Date) = p2.maxday 
AND MONTH(p1.Purchase_Date) = p2.maxmonth 
AND YEAR(p1.Purchase_Date) = 2021;

Top 20 buyers in 2021

In [ ]:
SELECT TOP (20) Buyer_ID, COUNT(Purchase_Date) AS Purchases_Time
FROM Purchases_Project..Purchases
GROUP BY Buyer_ID
ORDER BY 2 DESC;

Best selling items and wost selling items in 2021

In [ ]:
--best selling items
    --find the the total amount of purchase date for each item
/*
SELECT Product, COUNT(Purchase_Date) AS Highest_Purchases_Time
FROM Purchases_Project..Purchases
GROUP BY Product
ORDER BY 2 DESC;
*/
    --find the highest purchases times
WITH Total_time AS (
    SELECT Product, COUNT(Purchase_Date) AS Purchases_Time
    FROM Purchases_Project..Purchases
    GROUP BY Product
)
        --have to use a subquery table, then match the total time together
SELECT Total_time.Product, l.Item_Category, Purchases_Time
FROM Total_time,
     (SELECT MAX(Purchases_Time) AS Highest_Purchases_Time
      FROM Total_time) hp, Purchases_Project..Listing l
WHERE Total_time.Purchases_Time = hp.Highest_Purchases_Time
AND l.Product = Total_time.Product;



In [ ]:
--what is the percentage of those items in each category
WITH Total_time AS (
    SELECT Product, COUNT(Purchase_Date) AS Purchases_Time
    FROM Purchases_Project..Purchases
    GROUP BY Product
)
SELECT l.Item_Category,
       ((CONVERT(float, COUNT(Total_time.Product)) / 25.0) * 100) AS Category_Percentage,
       Total_time.Purchases_Time
FROM Total_time,
     (SELECT MAX(Purchases_Time) AS Highest_Purchases_Time
      FROM Total_time) hp, 
     Purchases_Project..Listing l
WHERE Total_time.Purchases_Time = hp.Highest_Purchases_Time
AND l.Product = Total_time.Product
GROUP BY l.Item_Category, Total_time.Purchases_Time;

In [ ]:
--Worst selling products (not being purchased in 2021 at all)
SELECT Product, Item_Category, COUNT(Purchase_Date) AS Purchases_Time 
FROM Purchases_Project..Listing
GROUP BY Product, Item_Category
HAVING COUNT(Purchase_Date) = 0;

In [ ]:
--what is the percentage of those items in each category
WITH worst_sell AS (
    SELECT Product, Item_Category, COUNT(Purchase_Date) AS Purchases_Time, 
           COUNT(Product) OVER() AS Total_Product 
    FROM Purchases_Project..Listing
    GROUP BY Product, Item_Category
    HAVING COUNT(Purchase_Date) = 0
)
SELECT Item_Category, 
       ROUND(((CONVERT(float, COUNT(Product))/Total_Product) * 100), 3) AS Category_Percentage,
       worst_sell.Purchases_Time
FROM worst_sell
GROUP BY Item_Category, Total_Product, worst_sell.Purchases_Time

Best selling and worst selling categories

In [ ]:
--Best selling categories
    --count the purchases time and group by category
/*
SELECT Item_Category, COUNT(Purchase_Date) AS Purchase_Time
FROM Purchases_Project..Listing
GROUP BY Item_Category
ORDER BY 2 DESC;
*/
WITH purchase_time AS (
    SELECT Item_Category, COUNT(Purchase_Date) AS Purchases_Time
    FROM Purchases_Project..Listing
    GROUP BY Item_Category
)
SELECT Item_Category, Purchases_Time
FROM purchase_time,
     (SELECT MAX(Purchases_Time) AS Highest_Purchases_Time
      FROM purchase_time) hp
WHERE purchase_time.Purchases_Time = hp.Highest_Purchases_Time;


In [ ]:
--Wost selling categories
WITH purchase_time AS (
    SELECT Item_Category, COUNT(Purchase_Date) AS Purchases_Time
    FROM Purchases_Project..Listing
    GROUP BY Item_Category
)
SELECT Item_Category, Purchases_Time
FROM purchase_time,
     (SELECT MIN(Purchases_Time) AS Lowest_Purchases_Time
      FROM purchase_time) hp
WHERE purchase_time.Purchases_Time = hp.Lowest_Purchases_Time;
--double check
/*
SELECT Item_Category, COUNT(Purchase_Date) AS Purchase_Time
FROM Purchases_Project..Listing
GROUP BY Item_Category
ORDER BY 2 ASC;
*/

What is average purchase ammount of each category?

In [ ]:
--join purchase and listing table then find avg
WITH info_table AS (
    SELECT l.Item_Category, p.Purchase_Amount
    FROM Purchases_Project..Purchases p
    FULL OUTER JOIN Purchases_Project..Listing l
        ON p.Product = l.Product
)
SELECT Item_Category, AVG(Purchase_Amount) AS Average_Amount
FROM info_table
GROUP BY Item_Category
ORDER BY 2 DESC


Highest rating items and lowest rating items in 2021

In [ ]:
--highest rating
SELECT Product, Item_Category, Subcategory_1, Subcategory_2,
       General_Rating AS Highest_Rating
FROM Purchases_Project..Listing
WHERE General_Rating = (SELECT MAX(General_Rating)
                        FROM Purchases_Project..Listing)
GROUP BY Product, Item_Category, Subcategory_1, Subcategory_2, General_Rating;


In [ ]:
--lowest rating
SELECT Product, Item_Category, Subcategory_1, Subcategory_2,
       General_Rating AS Lowest_Rating
FROM Purchases_Project..Listing
WHERE General_Rating = (SELECT MIN(General_Rating)
                        FROM Purchases_Project..Listing)
GROUP BY Product, Item_Category, Subcategory_1, Subcategory_2, General_Rating;

Best selling seller and worst selling seller

In [ ]:
--Best seller
/*
SELECT Seller_ID, COUNT(Purchase_Date) AS Purchases_Time
FROM Purchases_Project..Purchases
GROUP BY Seller_ID
ORDER BY 2 DESC;
*/
WITH Total_time AS (
    SELECT Seller_ID, COUNT(Purchase_Date) AS Highest_Purchased_Time
    FROM Purchases_Project..Purchases
    GROUP BY Seller_ID
)
        --have to use a subquery table, then match the total time together
SELECT Seller_ID, Highest_Purchased_Time
FROM Total_time,
     (SELECT MAX(Highest_Purchased_Time) AS High_Purchased_Time
      FROM Total_time) hp
WHERE Total_time.Highest_Purchased_Time = hp.High_Purchased_Time;

In [ ]:
--Worst seller
SELECT Seller_ID, COUNT(Purchase_Date) AS Lowest_Purchased_Time
FROM Purchases_Project..Listing
GROUP BY Seller_ID
HAVING COUNT(Purchase_Date) = 0;

In [ ]:
--for visualize
SELECT Seller_ID, COUNT(Purchase_Date) AS Purchased_Time
FROM Purchases_Project..Purchases
GROUP BY Seller_ID
ORDER BY 2 DESC;

Total revenue of each seller in 2021

In [ ]:
--total revenue of the year
SELECT SUM(Purchase_Amount) AS total_purchase
FROM Purchases_Project..Purchases;

--total seller
SELECT COUNT(Seller_ID) AS total_seller
FROM Purchases_Project..Listing


In [ ]:
--sum for all sellers who were bought from
/*
SELECT Seller_ID, SUM(Purchase_Amount) AS total_purchased
FROM Purchases_Project..Purchases
GROUP BY Seller_ID
ORDER BY 2 DESC;
*/

--sum for all sellers
    --join two table
WITH seller AS (
SELECT l.Seller_ID, p.Purchase_Amount, l.Purchase_Date
FROM Purchases_Project..Purchases p
FULL OUTER JOIN Purchases_Project..Listing l
    ON p.Product = l.Product
)
SELECT Seller_ID, SUM(ISNULL(Purchase_Amount, 0)) AS total_purchased
FROM seller
GROUP BY Seller_ID
ORDER BY 2;

 <span style="font-family: ArialMT; font-size: 11pt; background-color: rgb(255, 255, 255);">&nbsp;How many buyers purchased from more than 1 departments in 2021?</span>

In [ ]:
WITH buyer_table AS(
    SELECT p.Buyer_ID, l.Item_Category
    FROM Purchases_Project..Purchases p 
    JOIN Purchases_Project..Buyers b
        ON b.Buyer_ID = p.Buyer_ID
    JOIN Purchases_Project..Listing l 
        ON p.Product = l.Product
)
SELECT COUNT(*) AS Amount_Buyers 
FROM (SELECT Buyer_ID 
      FROM (SELECT Buyer_ID, COUNT(Buyer_ID) AS buyer 
            FROM buyer_table
            GROUP BY Buyer_ID) b
      WHERE b.buyer > 1) total;

 <span style="font-size: 11.000000pt; font-family: 'ArialMT'">&nbsp;What was the total purchase amount of all departments for buyers in 2021&nbsp;</span> <span style="font-family: ArialMT; font-size: 14.6667px;">who spent the highest amount in "clothing" ?</span>

In [ ]:
WITH purchase_table AS(
    SELECT p.Buyer_ID, SUM(p.Purchase_Amount) AS total_purchase
    FROM Purchases_Project..Purchases p
    JOIN Purchases_Project..Listing l
        ON p.Product = l.Product
        AND l.Item_Category LIKE '%clothing%'
    GROUP BY Buyer_ID) 
SELECT purchase_table.Buyer_ID, SUM(p.Purchase_Amount) AS total_purchase_amount
FROM purchase_table
JOIN Purchases_Project..Purchases p 
    ON purchase_table.Buyer_ID = p.Buyer_ID
WHERE purchase_table.total_purchase = (SELECT MAX(total_purchase)
                                       FROM purchase_table) 
GROUP BY purchase_table.Buyer_ID;